## IDEA <br>

- Use OMDB API: http://www.omdbapi.com/#usage
- Do the API request with the search, movies with word 'love', 2 pages
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page=2
- Like that get the list of titles of the movies (later create column in df)
- Do another API request title by title to get more info about the movies
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t=The+Shawshank+Redemption
- Check how to pass the movie name to the end of URL. Format: xx+xx+xxx+xx (words separated by '+')
- Put info from both requests to the df
- Scrap IMDB or Wikkipedia based on title or IMDB ID to get info from there
- Check if some data matching: Release year, Genre, Director, IMDB Rating
- Could also get more data about the director from Wiki or IMDB

## API <br>

- For the API portion of the project will need to make calls to your chosen API, successfully obtain a response, request data, convert it into a Pandas data frame, and export it as a CSV file.

## WEB SCRAPING <br>

- For the web scraping portion of the project, you will need to scrape the HTML from your chosen page, parse the HTML to extract the necessary information, and either save the results to a text (txt) file if it is text or into a CSV file if it is tabular data.

## 1. API

In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [16]:
#the function to flatten_json several times

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [3]:
# Doing the search title includes word 'love'

url= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page=2'
response = requests.get(url)
response

results = response.json()
results

<Response [200]>

{'Search': [{'Title': 'From Russia with Love',
   'Year': '1963',
   'imdbID': 'tt0057076',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BZjBiNGNlNmItZTk2Zi00YjRlLTk1NzEtNDI2YTNmN2EwNDhlXkEyXkFqcGdeQXVyNDY2MTk1ODk@._V1_SX300.jpg'},
  {'Title': 'From Paris with Love',
   'Year': '2010',
   'imdbID': 'tt1179034',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BODAwMDFjNjktMWY2Mi00MmVhLWI0MjYtNzg4OTI0NzA5YzBjXkEyXkFqcGdeQXVyNTIzOTk5ODM@._V1_SX300.jpg'},
  {'Title': 'I Love You Phillip Morris',
   'Year': '2009',
   'imdbID': 'tt1045772',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNDkzMjkwNjc5NV5BMl5BanBnXkFtZTcwNzEyOTk5Mw@@._V1_SX300.jpg'},
  {'Title': 'Love, Simon',
   'Year': '2018',
   'imdbID': 'tt5164432',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNTMyZDdiMzUtZjcxNS00Mjc3LTljY2UtYjI4YmY5NzJlYjc1XkEyXkFqcGdeQXVyMTA5OTkwNTc@._V1_SX300.jpg'},
  {'Title': 'Eat Pray Love',


In [18]:
#creating the list of title with the word 'love'

title_box = results.get('Search')
title_list = [title.get('Title') for title in title_box]
title_list

['From Russia with Love',
 'From Paris with Love',
 'I Love You Phillip Morris',
 'Love, Simon',
 'Eat Pray Love',
 'To Rome with Love',
 'Stuck in Love.',
 'A Lot Like Love',
 'Love Me If You Dare',
 'Wristcutters: A Love Story']

In [7]:
#the search to get the info about single movie
#need to sdee how to pass different movie title each time

url= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t=The+Shawshank+Redemption'
    
response = requests.get(url)
response

results = response.json()
results

flattened_data = json_normalize(results)
flattened_data

<Response [200]>

{'Title': 'The Shawshank Redemption',
 'Year': '1994',
 'Rated': 'R',
 'Released': '14 Oct 1994',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Director': 'Frank Darabont',
 'Writer': 'Stephen King (short story "Rita Hayworth and Shawshank Redemption"), Frank Darabont (screenplay)',
 'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler',
 'Plot': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Nominated for 7 Oscars. Another 21 wins & 35 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '90%'},
  {'Source': 'Metacritic', 'Value': '80/100'}],
 'Metascore': '80',
 'imdbRating': '9.3',
 'imdbVotes': '2,188,727',
 'imdbID': 'tt011116

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,...,80,9.3,"2,188,727",tt0111161,movie,27 Jan 1998,N/A,Columbia Pictures,N/A,True
